# Spin Glass NADE

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dinesh110598/Spinglass_NADE/blob/main/main.ipynb)

In [2]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.math as tfm
import numpy as np

If you're running on your local machine, simply download the library.py and TrainingData20k.npy files of the repo to the folder you're running this notebook on.

## Colab Instructions
Run the following cell if running on Google colaboratory notebook.

In [2]:
#!curl -o library.py https://raw.githubusercontent.com/dinesh110598/Spinglass_NADE/main/library.py
#!curl -o TrainingData20k.npy https://raw.githubusercontent.com/dinesh110598/Spinglass_NADE/main/TrainingData20k.npy

## Brief Introduction
We aim to construct a neural network (Neural Autoregressive Distribution Estimator) that can efficiently approximate the Boltzmann distribution of an EA spin glass system in equilibrium via the training data obtained using annealed MCMC simulations. See here for more details: https://arxiv.org/abs/2002.04292

The nitty gritty details of the neural network are coded in the library.py whose classes we'll be importing into this notebook directly:

In [3]:
from library import NADE_orig

In [4]:
model = NADE_orig (inshape=(20,20),num_hidden=20)

We have performed MCMC simulations of the EA lattice at T=0.5 externally and loading the numpy data for the same into here. Overall, we have 20000 latices of 20x20 EA lattices with Gaussian couplings to train our network with. In order to reduce the memory burden on the neural network, we made sure that *all latices have spins at (0,0) position = 1* and noting that multiplying the entire lattice by -1 gives an energetically equivalent configuration that the neural network need not learn

In [5]:
t_lattice = np.load ('TrainingData20k.npy')
np.all (t_lattice[:,0,0] == 1) #Checks if (0,0) spins=1

True

Here, we convert the NumPy array to a tf.data datset so that we can batch and loop over it conveniently while training

In [6]:
train_data = tf.data.Dataset.from_tensor_slices (t_lattice)
train_data = train_data.batch(20)

We'll call train_step method of the NADE_orig class on a sample of the training data. This step builds the "autograph" associated with the method to provide significant speedup to subsequent executions of the method

In [10]:
loss = model.train_step (tf.constant (t_lattice[0:20,:,:]))
tf.print (loss)

TensorShape([])

Though we can use fit() method available for keras models, we'll quickly write a custom training loop to keep things transparent and more flexible:

In [12]:
epochs = 1
for epoch in range (epochs):
    for step, data in enumerate(train_data):
        loss = model.train_step (data)
        if step%50 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step+1, float(loss))
            )
            print("%d samples seen so far" % ((step + 1) * 20))

Training loss (for one batch) at step 1: 0.6892
20 samples seen so far
Training loss (for one batch) at step 21: 0.6900
420 samples seen so far
Training loss (for one batch) at step 41: 0.6902
820 samples seen so far
Training loss (for one batch) at step 61: 0.6899
1220 samples seen so far
Training loss (for one batch) at step 81: 0.6898
1620 samples seen so far
